In [ ]:
!pip install selenium

In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

In [ ]:
!apt-get update
!apt-get install chromium chromium-driver

In [ ]:
!pip install undetected_chromedriver

In [ ]:
!pip install webdriver_manager
!pip install pyvirtualdisplay
!sudo apt-get install xvfb

In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service #as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
#from subprocess import CREATE_NO_WINDOW
from selenium.webdriver.chromium.webdriver import ChromiumDriver
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import undetected_chromedriver
from pyvirtualdisplay import Display

In [ ]:
display = Display(visible=0, size=(1920, 1200))  
display.start()
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('enable-automation')
options.add_argument('--disable-infobars')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-browser-side-navigation')
options.add_argument("--remote-debugging-port=9222")
# options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.add_argument("--log-level=3")
driver = webdriver.Chrome( options=options)

In [ ]:
url='https://www.glassdoor.com/Job/data-scientist-jobs-SRCH_KO0,14.htm?fromAge=1'
driver.get(url)
soup=BeautifulSoup(driver.page_source,"lxml")

In [ ]:
last_page = soup.find("div",class_="paginationFooter").text
print(last_page)

Page 1 of 6


In [ ]:
last_page = int(last_page.split()[-1])
last_page

6

In [ ]:
def scan():
    sleep(1)
    soup=BeautifulSoup(driver.page_source,"lxml")
    try:
        idd = soup.find("button",class_="gd-ui-button applyButton css-1wuyo1a evpplnh1").get("data-job-id")
    except:
        idd = "unknown"
    try:
        employerName = soup.find("div",class_="css-87uc0g").text
    except:
        employerName = "unknown"
    try:
        jobTitle = soup.find("div",class_="css-1vg6q84 e1tk4kwz4").text
    except:
        jobTitle = "unknown"
    try:
        location = soup.find("div",class_="css-56kyx5 e1tk4kwz5").text
    except:
        location = "unknown"
    try:
        detailSalary = driver.find_element(By.XPATH,"/html/body/div[2]/div/div/div/div/div[2]/section/div/div/article/div/div[1]/div/div/div[1]/div[3]/div[1]/div[4]/span").text
    except:
        detailSalary = "unknown"
    linkss = "unknown"
    try:    
        text2 = soup.find("div",class_="jobDescriptionContent").text
    except:
        text2 = "unknown"
    return([idd,jobTitle,employerName,detailSalary,location,linkss,text2])

In [ ]:
page = driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div/div[2]/section/article/div[2]/div/div[1]/button[7]')
knop= driver.find_element(By.XPATH,'/html/body/div[2]/div/div/div/div/div[2]/section/article/div[1]/ul/li[1]/div[2]/a')
sleep(2)
knop.click()
sleep(5)
try:
    xx = driver.find_element(By.XPATH,'/html/body/div[14]/div/div[2]/span')
    sleep(2)
    xx.click() 
except:
    pass

In [ ]:
data=[]
ids=[]
links=[]
for j in range(0,last_page):
    sleep(1)
    soup=BeautifulSoup(driver.page_source,"lxml")
    link = soup.find_all("a",class_="jobLink css-1rd3saf eigr9kq2")
    zid=soup.find_all("li",class_="react-job-listing")
    for ii in zid:
        ids.append(ii.get("data-id"))
    for l in link:
        ll = "https://www.glassdoor.com"+l.get("href")
        links.append(ll)
    
    for i in range(1,31):#31
        try:
            sleep(2)
            knop= driver.find_element(By.XPATH,f'/html/body/div[2]/div/div/div/div/div[2]/section/article/div[1]/ul/li[{i}]/div[2]/a')
            sleep(1)
            knop.click()
        except:
            print("eror klik")
            sleep(1)
        try:
            data.append(scan())
            print(data[-1])
        except:
            print("eror scan")
            sleep(1)
    page.click()
    sleep(1)

['unknown', 'Sr. Data Scientist', 'Humans Doing', '', 'Remote', 'unknown', "Our client is seeking a highly skilled and experienced Lead Data Scientist to support key client accounts and drive the development of data-driven solutions to their most significant business challenges. This will consist of managing and leading data strategy and analytics, creating data-driven solutions, developing and supporting team project deliverables, leading the development of internal data capabilities and infrastructure, and generally enabling the growth of the product and data business. The Lead Data Scientist will be responsible for leading a team of technical resources, working closely with business stakeholders to identify and prioritize opportunities for data-driven decision-making, and developing and implementing end-to-end solutions using statistical, machine learning, and data engineering techniques.Leading the execution, facilitation, and overall management of data workstreams on innovation pr

In [ ]:
for jj in range(len(data)):
    for zz in range(len(ids)):
        if data[jj][0] == ids[zz]:
            data[jj][5] = links[zz]
            #print(links[zz],data[jj][6],ids[zz],data[jj][0])

In [ ]:
len(links),len(ids)

(174, 174)

In [ ]:
df=pd.DataFrame(data,columns=['idd','jobTitle','employerName','detailSalary','location','links','text'])


In [ ]:
# driver.close()
driver.quit()

In [ ]:
df

,idd,jobTitle,employerName,detailSalary,location,links,text
0,unknown,Sr. Data Scientist,Humans Doing,,Remote,unknown,Our client is seeking a highly skilled and exp...
1,unknown,Survey Statistician - Health IT,Code360,,Remote,unknown,"Provide expertise regarding the planning, deve..."
2,unknown,Data Scientist,Vetro Tech Inc3.9,,"Seattle, WA",unknown,We are looking for a Data Scientist to analyze...
3,unknown,Data Scientist,Maddisoft,,"Houston, TX",unknown,Maddisoft has the following immediate opportun...
4,unknown,Data scientist,CEDENT,unknown,"Austin, TX",unknown,WORKER SKILLS AND QUALIFICATIONS (Required)\nB...
...,...,...,...,...,...,...,...
175,unknown,AI Prompt Engineer,DURNO CORP,,"Roy, UT",unknown,"Job Title: AI Prompt Engineer (Part-Time, Remo..."
176,unknown,AI Prompt Engineer,DURNO CORP,,"Roy, UT",unknown,"Job Title: AI Prompt Engineer (Part-Time, Remo..."
177,unknown,AI Prompt Engineer,DURNO CORP,,"Roy, UT",unknown,"Job Title: AI Prompt Engineer (Part-Time, Remo..."
178,unknown,AI Prompt Engineer,DURNO CORP,,"Roy, UT",unknown,"Job Title: AI Prompt Engineer (Part-Time, Remo..."
